In [1]:
import os
from dotenv import load_dotenv
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain_core import chat_history

load_dotenv("../../_apikeys.env")
api_key = os.getenv("DoogieOpenaiKey")
os.environ['OPENAI_API_KEY'] = api_key

In [ ]:
import gradio as gr

In [2]:
#
# Step 1-1: get test documents
#
import urllib.request

urllib.request.urlretrieve("https://github.com/chatgpt-kr/openai-api-tutorial/raw/main/ch07/2020_%EA%B2%BD%EC%A0%9C%EA%B8%88%EC%9C%B5%EC%9A%A9%EC%96%B4%20700%EC%84%A0_%EA%B2%8C%EC%8B%9C.pdf", filename="07_01_EcoKeyword.pdf")

('07_01_EcoKeyword.pdf', <http.client.HTTPMessage at 0x108edc9d0>)

In [2]:
#
# Step 1-2: load and split PDF
# 17~20 sec
#
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader( "07_01_EcoKeyword.pdf" )
pages = loader.load_and_split()  # about 17s
print('Load and Splits Docs: ', len(pages) )
#pages = loader.load()  # about 16s, include empty page


Load and Splits Docs:  366


In [ ]:
print('Load and Splits Docs: ', len(pages) )
print('some Docs[0]: ', pages[0].page_content[:100])
print('some Docs[5]: ', pages[5].page_content[:100])
print('some Docs[12]: ', pages[12].page_content[:100])
print('some Docs[13]: ', pages[13].page_content[:100])
print('some Docs[15]: ', pages[15].page_content[:100])

In [3]:
#
# Step 1-3: remove 1~12 and last of chunks
#
pageBodys = pages[13:]
print('pageBodys: ', len(pageBodys) )
#print('pageBodys[0]: ', pageBodys[0].page_content[:100]) # was 13
#print('pageBodys[2]: ', pageBodys[2].page_content[:100]) # was 15
#print('pageBodys[-1]: ', pageBodys[-1].page_content[:100]) # was last
pageBodys = pageBodys[:-1]
print('pageBodys: ', len(pageBodys) )
#print('pageBodys[-1]: ', pageBodys[-1].page_content[:100]) # was last

pageBodys:  353
pageBodys:  352


In [4]:
#
# Step2-1: using package 'langchain_openai', not 'OpenAIEmbeddings'
#          from langchain.embeddings import OpenAIEmbeddings
#          'langchain.embeddings' deprecated, use 'langchain_openai'
#
#from langchain.embeddings.openai import OpenAIEmbeddings
#from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings

myEmbeddings = OpenAIEmbeddings(model="text-embedding-3-large")
print("current OpenAIEmbeddings().model=", myEmbeddings.model)

persist_dir = "../localdb/my_chroma07_01"
myCollectionName = "my_collection"

current OpenAIEmbeddings().model= text-embedding-3-large


In [ ]:
#
# CAN NOT USE THIS STYLE with Over Token Max
# Step2-2: load chromaDB with pageBodys, by one Method
#
os.environ["CHROMA_TELEMETRY"] = "False"
#from langchain.vectorstores import Chroma
from langchain_chroma import Chroma

vdb = Chroma.from_documents(
    pageBodys,
    embedding=myEmbeddings,
    persist_directory=persist_dir,
    collection_name=myCollectionName
)

print(vdb._collection.count())

In [5]:
#
# Step2-2: load chromaDB with pageBodys, by splited Method
#
os.environ["CHROMA_TELEMETRY"] = "False"
#from langchain.vectorstores import Chroma
from langchain_chroma import Chroma

vdb = Chroma(
    embedding_function=myEmbeddings,
    persist_directory=persist_dir,
    collection_name=myCollectionName
)

def clear_before_addDoc():
    global vdb
    all_ids = vdb.get()["ids"]

    if ( len(all_ids) == 0 ):
        print(">> vdb size = 0 이므로 collection delete 하지 않음.")
        return

    # 컬렉션 내부 데이터 전체 삭제
    print(f">> vdb size = {len(all_ids)}, 벡터 컬렉션 데이터 삭제 시작...", end="")
    try:
        vdb._collection.delete(ids=all_ids)
        print("++벡터 컬렉션 데이터 삭제 완료.")
    except Exception as e:
        print("++벡터 컬렉션 삭제 실패:", e)
    print(">> vdb size = ", len( vdb.get()["ids"] ) )

def process_in_batches(documents, batch_size=100, reset_db=True):
    global vdb

    # -- 집어 넣을 문서 확인 --
    splitedDocsLen = len(documents)
    print(">> ready to insert splitedDocs, len = ", splitedDocsLen)

    if splitedDocsLen == 0:
        print(">> 처리할 splitedDocs가 비어있습니다. batch처리 중지...")
        return 0

    # -- 문서 크기를 batch_size로 나누어 확인하기
    batchCount = splitedDocsLen // batch_size
    if splitedDocsLen % batch_size > 0:
        batchCount += 1
    print(">> batchsize is ", batch_size, "and batchCount:", batchCount)

    # -- 문서를 batch_size로 잘라서 집어 넣기
    for i, batch in enumerate(range(0, splitedDocsLen, batch_size)):
        print(">>", i, "th batch, ", batch, "..", batch + batch_size - 1, end="")
        batchDocs = documents[batch:batch + batch_size]
        vdb.add_documents(batchDocs)
        print(" ++ added batchDocs len:", len(batchDocs))
    print(">> process_in_batch() >> vdb size:", len( vdb.get()["ids"] ))

print("[START] vdb 크기=", len( vdb.get()["ids"] ) )
print(">>Persist Dir:", persist_dir)
print(">>myCollectionName:", myCollectionName)
print(">> vdb._collectiob.count(): ",vdb._collection.count())

clear_before_addDoc()
process_in_batches(pageBodys)

print("[END] vdb 크기=", len( vdb.get()["ids"] ) )
print("[END] vdb._collectiob.count(): ",vdb._collection.count())

[START] vdb 크기= 352
>>Persist Dir: ../localdb/my_chroma07_01
>>myCollectionName: my_collection
>> vdb._collectiob.count():  352
>> vdb size = 352, 벡터 컬렉션 데이터 삭제 시작...++벡터 컬렉션 데이터 삭제 완료.
>> vdb size =  0
>> ready to insert splitedDocs, len =  352
>> batchsize is  100 and batchCount: 4
>> 0 th batch,  0 .. 99 ++ added batchDocs len: 100
>> 1 th batch,  100 .. 199 ++ added batchDocs len: 100
>> 2 th batch,  200 .. 299 ++ added batchDocs len: 100
>> 3 th batch,  300 .. 399 ++ added batchDocs len: 52
>> process_in_batch() >> vdb size: 352
[END] vdb 크기= 352
[END] vdb._collectiob.count():  352


In [6]:
print(">> vdb 크기=", len( vdb.get()["ids"] ) )
print(">> vdb._collection.count(): ",vdb._collection.count())
print(">> Persist Dir:", persist_dir)
print(">> myCollectionName:", myCollectionName)

for key in vdb._collection.get():
    #print(key, vdb._collection.get()[key][:100])
    #print(key, vdb.get()[key][:100])
    if ( vdb.get()[key] is None ):
        print(key, "is None")
    else:
        print(key, len(vdb.get()[key]))

>> vdb 크기= 352
>> vdb._collection.count():  352
>> Persist Dir: ../localdb/my_chroma07_01
>> myCollectionName: my_collection
ids 352
embeddings is None
documents 352
uris is None
included 2
data is None
metadatas 352


In [7]:
#
# Step 2-3: load documents from vdb(chromadb)
#
docsFromVdb = vdb.get()['documents']
print("chunk length of fromVdb: ", len(docsFromVdb) )
print("fromVdb[0]: ", docsFromVdb[0][:100])


chunk length of fromVdb:  352
fromVdb[0]:  1
ㄱ 
ㄱ
가계부실위험지수(HDRI)
가구의 소득 흐름은 물론 금융 및 실물 자산까지 종합적으로 고려하여 가계부채의 
부실위험을 평가하는 지표로, 가계의 채무상환능력을 소득 측면


In [ ]:
print("ids: ",          len(vdb.get()['ids']) )
print("ids: ",          len(vdb.get()['ids'][0]), ">>", vdb.get()['ids'][0] )
print("embeddings: ",   vdb.get()['embeddings'] )
print("uris: ",         vdb.get()['uris'] )
print("included: ",     vdb.get()['included'] )
print("data: ",         vdb.get()['data'] )
print("metadatas: ",    len( vdb.get()['metadatas']) )
print("metadatas: ",    vdb.get()['metadatas'][0] )
print("documents: ",    len( vdb.get()['documents']) )
print("documents: ",    vdb.get()['documents'][0][:100] )

In [ ]:
#
# Step2-4: get vdb's embeddings >> direct to 'embeddings' key
#
vdbEmbeddings = vdb.get(include=['embeddings'])['embeddings']
print('len embedding from vdb: ', len(vdbEmbeddings) )
print('len(vdbEmbeddings[0]): ', len(vdbEmbeddings[0]) )
print('vdbEmbeddings[0]: ', vdbEmbeddings[0][:100] )


In [9]:
#
# Step 3-1: retriever from vdb with question's similarity at RAG documents
# searchRetriever from vdb, is USE for RADIO chat's RAG
#
searchRetriever = vdb.as_retriever( search_kwargs={"k": 3} )
#searchResult = searchRetriever.get_relevant_documents("비트코인이 궁금해")
searchResult = searchRetriever.invoke("비트코인이 궁금해")

print('result docs: ',  len(searchResult) )
for i in range(len(searchResult)):
    print(f'result docs[{i}]: ',   searchResult[i].page_content[:100] )

result docs:  3
result docs[0]:  139
ㅂ 
비트코인
비트코인(bitcoin)은 가상통화(암호통화)이자 디지털 지급시스템이다. 비트코인 시스템
은 중앙 저장소 또는 단일 관리자가 없기 때문에 최초의 탈중앙화된 디
result docs[1]:  5
ㄱ 
가산금리
기준금리에 신용도 등의 차이에 따라 달리 덧붙이는 금리를 가산금리(또는 스프레드, 
spread)라고 한다. 예를 들어 은행이 대출금리를 결정할 때 고객의 신용위
result docs[2]:  136
경제금융용어 700선
간 대화의 깊이와 폭도 더욱 넓어지고 있다. 브릭스는 현재의 경제성장 속도와 앞으로의 
발전 전망에 비추어 신흥 경제대국으로 발돋움할 가능성이 높은 나


In [10]:
#
# Step 4-1: query to GPT, with RAG similar docs
#
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

myTemplate = """
당신은 한국은행에서 만든 금융용어를 설명해 주는 금융도우미 입니다.
Emma가 개발하여 만들었습니다.
주어진 검색 결과를 바탕으로 답변하세요.
검색 결과에 없는 내용이라면, 그래도 최대한 답변하세요.
{context}

Question" {question}
Answer:
"""
# myPrompt = PromptTemplate(input_variables=["context", "question"], template=myTemplate)
myPrompt = PromptTemplate.from_template(myTemplate)

myLLM = ChatOpenAI(
    model_name = "gpt-4o-mini",
    temperature=0.0,
)

myChain = RetrievalQA.from_chain_type(
    llm=myLLM,
    chain_type="stuff",  ###
    retriever=searchRetriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": myPrompt},
    # verbose=True, ###
)

In [11]:
question = "디커플링이란 무엇인가?"
#reChatChain = myChain({"question": question})
#reChatChain = myChain(question) # deprecate, old style
reChatChain = myChain.invoke(question)
print("resultChatbotChain size: ", len(reChatChain) )

resultChatbotChain size:  3


In [12]:
#
# Step 4-2: checkup result
#
print(reChatChain["query"])
print(reChatChain["result"])

for i in range(len(reChatChain["source_documents"])):
    print(f"src_docs[{i}].page: ", reChatChain["source_documents"][i].metadata["page_label"])
    print(f"src_docs[{i}].body: ", reChatChain["source_documents"][i].page_content[:100] )

디커플링이란 무엇인가?
디커플링(decoupling)은 탈동조화라는 의미로, 특정 국가나 지역의 경제가 인접한 다른 국가나 전반적인 세계 경제의 흐름과는 다른 양상을 보이는 현상을 말합니다. 예를 들어, 금융위기 이후 신흥국가나 유로지역 국가들이 미국 경제와 다른 모습을 보이는 경우가 디커플링의 한 예입니다. 또한, 주가, 금리, 환율 등 일부 경제 변수의 흐름이 국가 간 또는 특정 국가 내에서 서로 다른 방향으로 움직이는 경우도 디커플링으로 간주됩니다. 반대로, 한 나라 또는 지역의 경제가 인접한 다른 국가나 세계 경제의 흐름과 유사한 방향으로 움직이는 현상은 커플링(coupling)이라고 합니다.
src_docs[0].page:  115
src_docs[0].body:  98
경제금융용어 700선
가계와 기업이 소비와 투자를 미룸으로써 수요의 위축을 초래하여 디플레이션 압력을 
보다 크게 할 수도 있다. 이 같은 상황에서는 경제주체들의 물가상승률 
src_docs[1].page:  106
src_docs[1].body:  89
ㄷ 
유지한 채 원리금 수취권만을 양도하게 되어 있어 참가자가 대출채권에 대한 위험을 
부담하게 된다. 즉 금융기관이 채무자로부터 원리금을 회수하는 경우에 한하여 원리금을 

src_docs[2].page:  184
src_docs[2].body:  167
ㅅ 
감소로 비용이 낮아지며, 투명성도 향상되어 신뢰도가 높아짐으로써 더 안전하게 계약이 
실행될 수 있다. 예를 들면 손해보험 보상업무의 경우 보험가입자･보험사･손해사정인


In [13]:
question = "너는 누가 개발했는가?"
#question = "너는 무엇을 하는 챗봇이니?"
reChatChain = myChain.invoke(question)
print("result len: ", len(reChatChain["source_documents"]) )
print(reChatChain["result"])
for i in range(len(reChatChain["source_documents"])):
    print(f"src_docs[{i}].body: ", reChatChain["source_documents"][i].page_content[:100] )

result len:  3
저는 Emma가 개발한 금융도우미입니다.
src_docs[0].body:  139
ㅂ 
비트코인
비트코인(bitcoin)은 가상통화(암호통화)이자 디지털 지급시스템이다. 비트코인 시스템
은 중앙 저장소 또는 단일 관리자가 없기 때문에 최초의 탈중앙화된 디
src_docs[1].body:  103
ㄹ 
로보어드바이저
로보어드바이저(robo-advisor)는 로봇(robot)과 자문가(advisor)의 합성어이다. 이는 
인공지능 알고리즘, 빅데이터 등을 활용하여 투자
src_docs[2].body:  183
ㅇ 
아시아개발은행(ADB)
아시아･태평양지역의 경제성장 및 경제협력 촉진과 역내 개도국의 경제개발 지원을 
목적으로 1966년 8월 설립된 국제금융기구로서 필리핀 마닐라에


In [ ]:
question = "비트코인에 대해서 궁금하당~~?"
#question = "디커플링이란 무엇인가?"
#reChatChain = myChain(question)
reChatChain = myChain.invoke(question)
print("result len: ", len(reChatChain["source_documents"]) )
print(reChatChain["result"])
for i in range(len(reChatChain["source_documents"])):
    print(f"src_docs[{i}].body: ", reChatChain["source_documents"][i].page_content[:100] )

In [17]:
#
# gradio version check
# IMPORTANT: m
#
import gradio as gr
print(gr.__version__)

3.50.2


In [ ]:
#
# gr.ChatInterface() 테스트
#

import gradio as gr

def respond(message, history):
    return f"Echo: {message}"

demo = gr.ChatInterface(respond)
demo.launch()

In [ ]:
#
# simple version by ChatInterface() >> stable
#
import gradio as gr

def chat_response(message, history):
    """LangChain 챗봇 응답"""
    if not message.strip():
        return "질문을 입력해주세요."

    try:
        response = myChain.invoke({"query": message})
        return response["result"]
    except Exception as e:
        return f"❌ 오류: {str(e)}"

# 가장 간단한 인터페이스
demo = gr.ChatInterface(
    fn=chat_response,
    #type="messages",  # only for gradio 4.x
    title="📚 경제금융용어 챗봇",
    examples=["GDP란?", "인플레이션이란?"]
)

demo.launch()

In [62]:
#
# simple version by Block()
#
import gradio as gr

# 인터페이스를 생성.
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(
        label="😍😍🥰😎경제금융용어 챗봇",
        height=300
    )
    msg = gr.Textbox(
        label="⁉️질문해주세요!",
        placeholder="예: GDP란 무엇인가요?",
        lines=2
    )  # 하단의 채팅창의 레이블

    with gr.Row():
        submit = gr.Button("✅전송", variant="primary" )
        clear = gr.Button("☑️대화 초기화")  # 대화 초기화 버튼

    # 챗봇의 답변을 처리하는 함수
    def respond(message, chat_history):
        # 입력 검증
        if not message or not message.strip():
            return "", chat_history if chat_history else []
        # chat_history 초기화
        if chat_history is None:
            chat_history = []

        #bot_message = get_chatbot_response(message)
        #reChatChain = myChain.invoke( message )
        reChatChain = myChain.invoke({"query": message})
        bot_message = reChatChain["result"]

        # 채팅 기록에 사용자의 메시지와 봇의 응답을 추가.
        chat_history.append((message, bot_message))
        return message, chat_history

    def clear_chat():
        return []

    # 사용자의 입력을 제출(submit)하면 respond 함수가 호출.
    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    submit.click(respond, [msg, chatbot], [msg, chatbot])
    # '초기화' 버튼을 클릭하면 채팅 기록을 초기화.
    clear.click(clear_chat, None, chatbot, queue=False)

IMPORTANT: You are using gradio version 3.50.2, however version 4.44.1 is available, please upgrade.
--------


In [ ]:
# 인터페이스 실행.
demo.launch(
    debug=True,
    inline=True,
    share=True,
)

In [63]:
#
# interim test for chating history
# chat_hist는 누적된 대화를 질문과 대답 2개의 dictionary 이다.
#
chat_hist = []

In [64]:
test_msg = "GDP란?"
result = myChain.invoke({"query": test_msg})
print(f"Type: {type(result)}")
#print(f"Content: {result}")
print(f"Content: {result['result'][:100]} ...+++ ")

Type: <class 'dict'>
Content: 국내총생산(GDP; Gross Domestic Product)은 한 나라의 영역 내에서 가계, 기업, 정부 등 모든 경제 주체가 일정 기간 동안 생산활동에 참여하여 창출한 부가가치 ...+++ 


In [65]:
new_msg, new_hist = respond(test_msg, chat_hist)
print(f"New message: {new_msg} <- {test_msg}")
print(f"New history: {new_hist}")

New message: GDP란? <- GDP란?
New history: [('GDP란?', '국내총생산(GDP; Gross Domestic Product)은 한 나라의 영역 내에서 가계, 기업, 정부 등 모든 경제 주체가 일정 기간 동안 생산활동에 참여하여 창출한 부가가치 또는 최종 생산물을 시장가격으로 평가한 합계입니다. 여기에는 국내에 거주하는 비거주자(외국인)에게 지급되는 소득도 포함됩니다. \n\nGDP는 가격의 적용 방법에 따라 명목 GDP(Nominal GDP)와 실질 GDP(Real GDP)로 구분됩니다. 명목 GDP는 국가 경제의 규모나 구조 등을 파악하는 데 사용되며, 실질 GDP는 경제 성장, 경기 변동 등 전반적인 경제 활동의 흐름을 분석하는 데 이용됩니다. \n\nGDP는 경제의 전반적인 성과를 측정하는 중요한 지표로, 국가의 경제적 건강 상태를 평가하는 데 필수적인 역할을 합니다.')]


In [66]:
test_msg = "너는 누가 개발했는가?"
result = myChain.invoke({"query": test_msg})
print(f"Type: {type(result)}")
#print(f"Content: {result}")
print(f"Content: {result['result'][:100]} ...+++ ")

Type: <class 'dict'>
Content: 저는 Emma가 개발한 금융도우미입니다. ...+++ 


In [67]:
new_msg, new_hist = respond(test_msg, chat_hist)
print(f"New message: {new_msg} <- {test_msg}")
print(f"New history: {new_hist}")

New message: 너는 누가 개발했는가? <- 너는 누가 개발했는가?
New history: [('GDP란?', '국내총생산(GDP; Gross Domestic Product)은 한 나라의 영역 내에서 가계, 기업, 정부 등 모든 경제 주체가 일정 기간 동안 생산활동에 참여하여 창출한 부가가치 또는 최종 생산물을 시장가격으로 평가한 합계입니다. 여기에는 국내에 거주하는 비거주자(외국인)에게 지급되는 소득도 포함됩니다. \n\nGDP는 가격의 적용 방법에 따라 명목 GDP(Nominal GDP)와 실질 GDP(Real GDP)로 구분됩니다. 명목 GDP는 국가 경제의 규모나 구조 등을 파악하는 데 사용되며, 실질 GDP는 경제 성장, 경기 변동 등 전반적인 경제 활동의 흐름을 분석하는 데 이용됩니다. \n\nGDP는 경제의 전반적인 성과를 측정하는 중요한 지표로, 국가의 경제적 건강 상태를 평가하는 데 필수적인 역할을 합니다.'), ('너는 누가 개발했는가?', '저는 Emma가 개발한 금융도우미입니다.')]
